# A TSP Example

Find the closed tour that visits all ten cities and has the minimum total length. Distances between pairs of cities are given in the table below:

|         |  ATL  |  ORD  |  DEN  |  IAH  |  LAX  |  MIA  |  JFK  |  SFO  |  SEA  |  DCA  |
|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|**ATL**  |    0  |  587  | 1212  |  701  | 1936  |  604  |  748  | 2139  | 2182  |  543  |
|**ORD**  |  587  |    0  |  920  |  940  | 1745  | 1188  |  713  | 1858  | 1737  |  597  |
|**DEN**  | 1212  |  920  |    0  |  879  |  831  | 1726  | 1631  |  949  | 1021  | 1494  |
|**IAH**  |  701  |  940  |  879  |    0  | 1379  |  968  | 1420  | 1645  | 1891  | 1220  |
|**LAX**  | 1936  | 1745  |  831  | 1379  |    0  | 2339  | 2451  |  347  |  959  | 2300  |
|**MIA**  |  604  | 1188  | 1726  |  968  | 2339  |    0  | 1092  | 2594  | 2734  |  923  |
|**JFK**  |  748  |  713  | 1631  | 1420  | 2451  | 1092  |    0  | 2571  | 2408  |  205  |
|**SFO**  | 2139  | 1858  |  949  | 1645  |  347  | 2594  | 2571  |    0  |  678  | 2442  |
|**SEA**  | 2182  | 1737  | 1021  | 1891  |  959  | 2734  | 2408  |  678  |    0  | 2329  |
|**DCA**  |  543  |  597  | 1494  | 1220  | 2300  |  923  |  205  | 2442  | 2329  |    0  |

In [24]:
# Define the problem data (cities and costs)
using JuMP, NamedArrays, HiGHS

cities = [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]

distances = [     0   587  1212   701  1936   604   748  2139  2182   543
                587     0   920   940  1745  1188   713  1858  1737   597
               1212   920     0   879   831  1726  1631   949  1021  1494
                701   940   879     0  1379   968  1420  1645  1891  1220
               1936  1745   831  1379     0  2339  2451   347   959  2300
                604  1188  1726   968  2339     0  1092  2594  2734   923
                748   713  1631  1420  2451  1092     0  2571  2408   205
               2139  1858   949  1645   347  2594  2571     0   678  2442
               2182  1737  1021  1891   959  2734  2408   678     0  2329
                543   597  1494  1220  2300   923   205  2442  2329     0  ]

c = NamedArray(distances,(cities,cities))
N = size(distances,1);

![TSP0](plot0.png)

## Solve TSP using min-cost flow relaxation

In [25]:
# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in cities
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = cities
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end

# HELPER FUNCTION FOR MAPPING AIRPORTS

data = [ 33.636700  -84.427863 
         41.977320  -87.908005
         39.861667 -104.673166
         29.645417  -95.278888
         33.942437 -118.408121
         25.795361  -80.290110
         40.639926  -73.778694
         37.618806 -122.375416
         47.449889 -122.311777
         38.851444  -77.037721 ]
lat = Dict(zip(cities,data[:,1]))
lon = Dict(zip(cities,data[:,2]))

Dict{Symbol, Float64} with 10 entries:
  :ATL => -84.4279
  :SFO => -122.375
  :DEN => -104.673
  :DCA => -77.0377
  :SEA => -122.312
  :ORD => -87.908
  :IAH => -95.2789
  :LAX => -118.408
  :JFK => -73.7787
  :MIA => -80.2901

In [26]:
# solve the simplest version (min-cost flow version; it's an assignment problem)

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, 0 <= x[cities,cities] <= 1)                               # LP relaxation
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)    # exacly one edge out of each node
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)    # exactly one edge into each node
@constraint(m, c3[i in cities], x[i,i] == 0 )                          # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))  # minimize total cost
optimize!(m)

# pretty print the solution
xx = value.(x)
sol = NamedArray(zeros(Int,N,N),(cities,cities))
for i in cities
    for j in cities
        if xx[i,j] > 0.5
            sol[i,j] = 1
        end
    end
end
println(sol)
sleep(1)
display(getAllSubtours(sol))
;

5-element Vector{Any}:
 [:ATL, :MIA, :ATL]
 [:ORD, :IAH, :ORD]
 [:DEN, :SEA, :DEN]
 [:LAX, :SFO, :LAX]
 [:JFK, :DCA, :JFK]

10×10 Named Matrix{Int64}
A ╲ B │ :ATL  :ORD  :DEN  :IAH  :LAX  :MIA  :JFK  :SFO  :SEA  :DCA
──────┼───────────────────────────────────────────────────────────
:ATL  │    0     0     0     0     0     1     0     0     0     0
:ORD  │    0     0     0     1     0     0     0     0     0     0
:DEN  │    0     0     0     0     0     0     0     0     1     0
:IAH  │    0     1     0     0     0     0     0     0     0     0
:LAX  │    0     0     0     0     0     0     0     1     0     0
:MIA  │    1     0     0     0     0     0     0     0     0     0
:JFK  │    0     0     0     0     0     0     0     0     0     1
:SFO  │    0     0     0     0     1     0     0     0     0     0
:SEA  │    0     0     1     0     0     0     0     0     0     0
:DCA  │    0     0     0     0     0     0     1     0     0     0
0; Iter: Time   4.053e-08; average =   4.053e-09; Bound =   4.138e-06


![TSP1](plot1.png)

## Solve TSP using adaptive subtour elimination

In [27]:
m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[cities,cities], Bin)                                       # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)     # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)     # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                        # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost

sols = []
# we'll run the heuristic 30 times and hope we get an optimal solution
for iters = 1:6
    optimize!(m)
    # total  length of current tour
    println("Tour length: ", objective_value(m))
    xx = value.(x) # save solution
    sol = NamedArray(zeros(Int,N,N),(cities,cities))
    for i in cities
        for j in cities
            if xx[i,j] > 0.5
                sol[i,j] = 1
            end
        end
    end
    push!(sols,sol) # save solution
    subtours = getAllSubtours(sol)  # get all the subtours
    display(subtours) 
    sleep(1)
    # get length of the subtour list
    len = length(subtours)
    if len == 1                    
        # solution is just a single tour!
        println("SOLVED!")
        break
    else
        for subtour in subtours
            L = length(subtour)
            # add constraints that cut off each subtour in the list (add two for each subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end

5-element Vector{Any}:
 [:ATL, :MIA, :ATL]
 [:ORD, :IAH, :ORD]
 [:DEN, :SEA, :DEN]
 [:LAX, :SFO, :LAX]
 [:JFK, :DCA, :JFK]

3-element Vector{Any}:
 [:ATL, :IAH, :MIA, :ATL]
 [:ORD, :DCA, :JFK, :ORD]
 [:DEN, :LAX, :SFO, :SEA, :DEN]

3-element Vector{Any}:
 [:ATL, :MIA, :DCA, :JFK, :ORD, :ATL]
 [:DEN, :IAH, :DEN]
 [:LAX, :SFO, :SEA, :LAX]

3-element Vector{Any}:
 [:ATL, :DCA, :JFK, :ORD, :IAH, :MIA, :ATL]
 [:DEN, :LAX, :DEN]
 [:SFO, :SEA, :SFO]

3-element Vector{Any}:
 [:ATL, :ORD, :JFK, :DCA, :ATL]
 [:DEN, :SFO, :LAX, :SEA, :DEN]
 [:IAH, :MIA, :IAH]

1-element Vector{Any}:
 [:ATL, :MIA, :IAH, :LAX, :SFO, :SEA, :DEN, :ORD, :JFK, :DCA, :ATL]

Tour length: 6234.0
Tour length: 6665.0
Tour length: 6774.0
Tour length: 6991.000000000019
Tour length: 7260.0
Tour length: 7377.999999999284
SOLVED!
0; Iter: Time   3.815e-08; average =   3.815e-09; Bound =   3.895e-06
0; Iter: Time    4.53e-08; average =    4.53e-09; Bound =   4.625e-06
0; Iter: Time   3.338e-08; average =   3.338e-09; Bound =   3.408e-06
0; Iter: Time   3.576e-08; average =   3.576e-09; Bound =   3.651e-06
0; Iter: Time   3.338e-08; average =   3.338e-09; Bound =   3.408e-06
0; Iter: Time   3.815e-08; average =   3.815e-09; Bound =   3.895e-06


![TSP3](plot3.png)

## Miller-Tucker-Zemlin formulation

In [28]:
m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[cities,cities], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)      # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)      # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost
                                    
# MTZ variables and constraints
@variable(m, u[cities])
@constraint(m, c4[i in cities, j in cities[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
sol = NamedArray(zeros(Int,N,N),(cities,cities))
    for i in cities
        for j in cities
            if xx[i,j] > 0.5
                sol[i,j] = 1
            end
        end
    end
subtours = getAllSubtours(sol) 
display(subtours)
println("Tour length: ", objective_value(m))

1-element Vector{Any}:
 [:ATL, :MIA, :IAH, :LAX, :SFO, :SEA, :DEN, :ORD, :JFK, :DCA, :ATL]

Tour length: 7378.0
0; Iter: Time   3.815e-08; average =   3.815e-09; Bound =   3.895e-06


![TSP4](plot4.png)

